In [1]:
import numpy as np
from astropy.io import fits
import cv2
import os
import math

In [2]:
def min_max(x, ax, ay):
    min_x = x.min()
    max_x = x.max()
    a = np.zeros((ax, ay))
    b = np.zeros((ax, ay))
    z = np.zeros((ax, ay))
    for i in range(ax):
        for j in range(ay):
            z[i][j] = (x[i][j] - min_x)/(max_x - min_x) * 255
            a[i][j],b[i][j] = math.modf(z[i][j])

    return a,b

In [6]:
def image_conv(image):
    i = fits.getdata(image)
    cv2.imwrite("test.tif",i.astype('uint16'))
    c = cv2.imread("test.tif",-1)

    return c

In [7]:
# FITS画像ファイルのリストを作成
# image_files = ["../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2491939184_b.fit",
#                "../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2491942076_v.fit",
#                "../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2491945018_w.fit",
#                  "../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2491947926_p.fit"]  # すべての画像ファイルを含むリスト

# image_files = ["../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489428266_b.fit",
#                "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489432123_v.fit",
#                "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489436012_w.fit",
#                  "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489439885_p.fit"]

image_files = ["../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2490234689_b.fit",
               "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2490238562_v.fit",
               "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2490242451_w.fit",
               "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2490246307_p.fit"]


# 出力ディレクトリとファイル名のプレフィックスを指定
home_dir = os.path.expanduser("~")
output_directory = os.path.join(home_dir, "Documents/after_graduate2/output_directory_op/")
file_prefix = 'registered_image_'  # ファイル名のプレフィックスを指定

hdulist = fits.open(image_files[2])  
hdu = hdulist[0]
data = hdu.data
header = hdu.header
x_axis = header["NAXIS1"]
y_axis = header["NAXIS2"]

name_prefix = []
for i,name in enumerate(image_files):
    parts = name.split(".f")
    part = parts[0].split("/data/")
    par = part[1].split("/st")
    p = f"{par[0]}{par[1]}_reg_"
    name_prefix.append(p)

name = image_files[0]
parts = name.split(".f")
part = parts[0].split("/data/")
par = part[1].split("/st")
p = f"{par[0]}"
name_prefix.append(p)

# 最初の画像を基準画像として読み込み
images = []
#少数部分の保存
float_images = []
f = []
images_uint16 = []
for i,filename in enumerate(image_files):
    images_uint16.append(image_conv(filename))
    f.append(fits.getdata(filename))
    a,b = min_max(f[i], x_axis, y_axis)
    if i == 2:
      reference_image = np.uint8(b)
    images.append(np.uint8(b))
    float_images.append(a)


In [85]:
print(fits.getdata(image_files[0]).min())
print(reference_image.min())
print(data)
print(min_max_decode(reference_image, data, x_axis, y_axis))


96
0
[[2144 1898 1951 ... 3182 3228 3164]
 [2349 2083 2131 ... 3256 3196 3177]
 [2387 2140 2189 ... 3291 3221 3125]
 ...
 [3392 3088 2873 ... 2977 2855 2561]
 [3257 3267 3309 ... 2914 2736 2557]
 [3053 3243 3306 ... 2980 2956 2897]]
[[2137. 1888. 1940. ... 3173. 3225. 3160.]
 [2347. 2072. 2124. ... 3251. 3186. 3173.]
 [2386. 2137. 2176. ... 3278. 3212. 3120.]
 ...
 [3383. 3081. 2871. ... 2976. 2845. 2557.]
 [3251. 3265. 3304. ... 2911. 2727. 2543.]
 [3042. 3238. 3304. ... 2976. 2950. 2884.]]


In [9]:
output_directory = os.path.join(home_dir, "Documents/after_graduate2/output_perspective/")
registered_images = []


# 基準画像から他の画像への変換行列を計算し、変換を適用
for i,image_file in enumerate(images):
    
    # 画像の特徴点を検出
    akaze = cv2.AKAZE_create()
    keypoints1, descriptors1 = akaze.detectAndCompute(reference_image, None)
    keypoints2, descriptors2 = akaze.detectAndCompute(image_file, None)
    
    # 特徴点マッチングと変換行列の計算
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors2, descriptors1, k = 2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append([m])
    
    #適切なキーポイントを選択
    src_pts = np.float32([keypoints2[m[0].queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints1[m[0].trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    H, status = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    # 変換行列を適用して画像を基準画像の座標系に変換
    registered_image = cv2.warpPerspective(images_uint16[i], H, (image_file.shape[1], image_file.shape[0]))
    
    registered_images.append(registered_image)

for i in range(len(registered_images)):
    output_jpgname = f"{output_directory}{name_prefix[i]}.jpg"
    output_filename = f"{output_directory}{name_prefix[i]}.fits"
    cv2.imwrite(output_jpgname, registered_images[i].astype('uint8'))
    fits.writeto(output_filename, registered_images[i], overwrite=True)



In [5]:
output_directory = os.path.join(home_dir, "Documents/after_graduate2/output_perspective_test/")
registered_images = []


# 基準画像から他の画像への変換行列を計算し、変換を適用
for i,image_file in enumerate(images):
    
    # 画像の特徴点を検出
    akaze = cv2.AKAZE_create()
    keypoints1, descriptors1 = akaze.detectAndCompute(reference_image, None)
    keypoints2, descriptors2 = akaze.detectAndCompute(image_file, None)
    
    # 特徴点マッチングと変換行列の計算
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors2, descriptors1, k = 2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append([m])
    
    #適切なキーポイントを選択
    src_pts = np.float32([keypoints2[m[0].queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints1[m[0].trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    M, inliers = cv2.estimateAffine2D(src_pts, dst_pts)
    
    # 変換行列を適用して画像を基準画像の座標系に変換
    registered_image = cv2.warpAffine(image_file, M, (image_file.shape[1], image_file.shape[0]))
    
    registered_images.append(registered_image)

for i in range(len(registered_images)):
    output_jpgname = f"{output_directory}{name_prefix[i]}.jpg"
    output_filename = f"{output_directory}{name_prefix[i]}.fits"
    cv2.imwrite(output_jpgname, registered_images[i])
    #fits出力用に,正規化した時にでた少数部分を足し合わせて正規化の逆の計算をし元画像と値が近くなるようにする
    reg_flo = []
    reg_flo = min_max_decode(registered_images[i].astype(float) + float_images[i], f[i], x_axis, y_axis)
    fits.writeto(output_filename, registered_images[i], overwrite=True)

In [155]:
print(keypoints1)
print(M)

[<KeyPoint 0x7ff2b6ced1e0>, <KeyPoint 0x7ff2b1478f30>, <KeyPoint 0x7ff2b6c48930>, <KeyPoint 0x7ff2b6c487e0>, <KeyPoint 0x7ff2b6c488d0>, <KeyPoint 0x7ff2b6c48f60>, <KeyPoint 0x7ff2b6c48cc0>, <KeyPoint 0x7ff2b6c486c0>, <KeyPoint 0x7ff2b6c48960>, <KeyPoint 0x7ff2b6c486f0>, <KeyPoint 0x7ff2b6c48660>, <KeyPoint 0x7ff2b6c48a80>, <KeyPoint 0x7ff2b6c48f00>, <KeyPoint 0x7ff2b6c9d9f0>, <KeyPoint 0x7ff2b6c9da80>, <KeyPoint 0x7ff2b6c9d960>, <KeyPoint 0x7ff2b6c9d5d0>, <KeyPoint 0x7ff2b6c9df90>, <KeyPoint 0x7ff2b6c9d510>, <KeyPoint 0x7ff2b6c9dd80>, <KeyPoint 0x7ff2b6c9dd50>, <KeyPoint 0x7ff2b6c9d060>, <KeyPoint 0x7ff2b6c9d810>, <KeyPoint 0x7ff2b6c9dd20>, <KeyPoint 0x7ff2b6c9dea0>, <KeyPoint 0x7ff2b6c9d4e0>, <KeyPoint 0x7ff2b6c9d660>, <KeyPoint 0x7ff2b6c9dfc0>, <KeyPoint 0x7ff2b6c9d120>, <KeyPoint 0x7ff2b6c9d090>, <KeyPoint 0x7ff2b72662d0>, <KeyPoint 0x7ff2b7266120>, <KeyPoint 0x7ff2b7266270>, <KeyPoint 0x7ff2b72661e0>, <KeyPoint 0x7ff2b7266210>, <KeyPoint 0x7ff2b7266300>, <KeyPoint 0x7ff2b72664b0>, 

In [108]:
print(reg_flo)
print(f[3])

[[1095. 1873. 1816. ... 2705. 2708. 2338.]
 [1147. 2003. 1918. ... 2741. 2737. 2388.]
 [1167. 2049. 1963. ... 2759. 2706. 2356.]
 ...
 [ 669.  668.  669. ...  663.  670.  662.]
 [ 663.  665.  661. ...  662.  670.  660.]
 [ 663.  662.  667. ...  665.  663.  668.]]
[[ 660  774 1022 ... 2618 2556 2556]
 [ 734  839 1124 ... 2459 2455 2508]
 [ 874 1081 1648 ... 2487 2315 2313]
 ...
 [2432 2562 2846 ... 2764 2814 2817]
 [2492 2548 2729 ... 2828 2814 2859]
 [2688 2806 2822 ... 2907 2894 2954]]


In [142]:
col_img = []
col_img = cv2.merge((registered_images[0],registered_images[1],registered_images[2]))
name = image_files[0]
parts = name.split(".f")
part = parts[0].split("/data/")
par = part[1].split("/st")
col_name = "_color"
p = f"{par[0]}{col_name}"
name_prefix.append(p)
output_coljpgname = f"{output_directory}{name_prefix[4]}_2.jpg"
cv2.imwrite(output_coljpgname, col_img)

True

In [159]:
affine_matrix = M

# アフィン変換行列から回転の角度、平行移動、拡大倍率を取得
angle = np.arctan2(affine_matrix[1, 0], affine_matrix[0, 0])
translation_x = affine_matrix[0, 2]
translation_y = affine_matrix[1, 2]
scale_x = affine_matrix[0, 0]
scale_y = affine_matrix[1, 1]

# 角度を度数法に変換
angle_degrees = np.degrees(angle)

cx = translation_x / np.cos(angle)
cy = translation_y / np.sin(angle)

# 結果
print("回転角度（度数法）:", angle_degrees)
print("X方向の平行移動:", translation_x)
print("Y方向の平行移動:", translation_y)
print("X方向の拡大倍率:", scale_x)
print("Y方向の拡大倍率:", scale_y)
print("回転中心のX座標:", cx)
print("回転中心のY座標:", cy)

回転角度（度数法）: 0.10421876184939545
X方向の平行移動: -1.9579941654023187
Y方向の平行移動: 1.778319978824197
X方向の拡大倍率: 0.9942943214010063
Y方向の拡大倍率: 1.0024314931226048
回転中心のX座標: -1.9579974045335997
回転中心のY座標: 977.6578016120237
